In [ ]:
import re
import requests
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup

url = 'https://www.crunchyroll.com/sitemap'
request = requests.get(url)
soup = BeautifulSoup(request.content)

sitemap = soup.findAll('loc')

languages_symbols = ['en-gb', 'es', 'es-es', 'pt-br', 'pt-pt', 'fr', 'de', 'ar', 'it', 'ru']
languages_symbols_url = [ f'https://www.crunchyroll.com/{i}' for i in languages_symbols]

anime_links = []
for url in tqdm(sitemap):
    request = requests.get(url.text)
    soup = BeautifulSoup(request.content)
    
    for link in soup.findAll('loc'):
        link = link.text

        # Looks only urls with len greater than https://www.crunchyroll.com/
        if len(link) > 28:
            if not link in languages_symbols_url:
                re_search_lang = re.search(r'https://www\.crunchyroll\.com/([a-z]{2}|[a-z]{2}-[a-z]{2})/', link)
                re_search_tabs = re.search(r'(https://www\.crunchyroll\.com/.*)/', link)
                if not re_search_lang and not re_search_tabs:
                    anime_links.append(link)

 17%|█████████████▋                                                                    | 11/66 [02:10<12:07, 13.23s/it]

In [ ]:
len(anime_links)

In [ ]:
anime_links[:20]